In [6]:
from project.thermal import conventional_heating_final, heating_consumption, HDD_EQ
from project.input.param import generic_input
import pandas as pd

In [2]:
ratio_surface = generic_input['ratio_surface']
ratio_surface

,Wall,Floor,Roof,Windows
Housing type,,,,
Single-family,1.2,0.96,0.98,0.15
Multi-family,0.9,0.00,0.00,0.15


In [3]:
buildings = {'Renovated': [0.2, 0.2, 0.2, 1.6, 0.8],
             'Old': [2, 2, 2, 4, 0.7],
             'New': [0.1, 0.1, 0.1, 1.2, 3]
             }

buildings = pd.DataFrame(buildings, index=pd.Index(['Wall', 'Floor', 'Roof', 'Windows', 'Efficiency'])).T
buildings = pd.concat([buildings] * 2, keys=['Single-family', 'Multi-family'], names=['Housing type'])
buildings

Wall  Floor  Roof  Windows  Efficiency
Housing type                                                   
Single-family Renovated   0.2    0.2   0.2      1.6         0.8
              Old         2.0    2.0   2.0      4.0         0.7
              New         0.1    0.1   0.1      1.2         3.0
Multi-family  Renovated   0.2    0.2   0.2      1.6         0.8
              Old         2.0    2.0   2.0      4.0         0.7
              New         0.1    0.1   0.1      1.2         3.0

In [4]:
conventional_heating_final(buildings['Wall'], buildings['Floor'], buildings['Roof'], buildings['Windows'], ratio_surface, buildings['Efficiency'])

Housing type            
Single-family  Renovated     68.159817
               Old          431.752224
               New           13.018548
Multi-family   Renovated     36.980527
               Old          177.386247
               New            7.556878
dtype: float64

In [7]:
heating_consumption(buildings['Wall'], buildings['Floor'], buildings['Roof'], buildings['Windows'], buildings['Efficiency'], ratio_surface, HDD_EQ)

Housing type            
Single-family  Renovated     78.991507
               Old          583.590526
               New           14.098636
Multi-family   Renovated     45.082924
               Old          254.008262
               New            8.878821
Name: Consumption, dtype: float64